In [1]:
#returns the html content of given page
def render(source_html):
    #Fully render HTML, JavaScript and all
    import sys
    from PyQt5.QtCore import QEventLoop
    from PyQt5.QtWidgets import QApplication
    from PyQt5.QtWebEngineWidgets import QWebEngineView

    class Render(QWebEngineView):
        def __init__(self, html):
            self.html = None
            self.app = QApplication(sys.argv)
            QWebEngineView.__init__(self)
            self.loadFinished.connect(self._loadFinished)
            self.setHtml(html)
            while self.html is None:
                self.app.processEvents(QEventLoop.ExcludeUserInputEvents | QEventLoop.ExcludeSocketNotifiers | QEventLoop.WaitForMoreEvents)
            self.app.quit()

        def _callable(self, data):
            self.html = data

        def _loadFinished(self, result):
            self.page().toHtml(self._callable)

    return Render(source_html).html

#Extract all the content from td tags and check for rows with no nulls values and len(cols)=count
def extract_data(souptable,col_count): 
    arr=[]    
    for row in souptable.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) == col_count:
            arr.append([])
            for c in range(len(cols)):
                value=cols[c].find(text=True)
                arr[-1].append( value )
    return arr

import requests
import pandas as pd
from bs4 import BeautifulSoup

#Open the options URL
hist_html = requests.get("https://finance.yahoo.com/quote/AMZN/history?p=AMZN").text
#Genrate the text and parse html and find 'table' tag
rendered_hist_html = render(hist_html)
soup_h = BeautifulSoup(rendered_hist_html, 'html.parser')
hist_souptable = soup_h.find('table')
columns = [ "Date",
           "Open",
           "High",
           "Low",
           "Close",
           "Adj Close",
           "Volume"
          ]
#Extract data method return the values scraped from the yahoo page
hist_data=extract_data(hist_souptable,7)

#print the sample data that is generated
hist_df = pd.DataFrame(hist_data, columns=columns )
print(hist_df.head())

#Write the Options data to opts.csv
hist_df.to_csv("hist.csv",index=False)



           Date      Open      High       Low     Close Adj Close     Volume
0  Mar 12, 2018  1,592.60      0.00      0.00  1,599.55  1,599.55          -
1  Mar 09, 2018  1,563.50  1,578.94  1,559.08  1,578.89  1,578.89  4,436,100
2  Mar 08, 2018  1,550.00  1,554.88  1,545.25  1,551.86  1,551.86  3,862,600
3  Mar 07, 2018  1,526.52  1,545.90  1,522.51  1,545.00  1,545.00  4,174,100
4  Mar 06, 2018  1,533.20  1,542.13  1,528.00  1,537.64  1,537.64  4,561,700
